This requires you to write a 2D GAN game. I let you to get into the topic yourself, whitout any explonations from my side. You can watch lecture, seminar, read papers and tutorials (fun, fun, fun).

### Homework

I want you to implement a simple 2D GAN game. The kind of animation, I want to see is like in [this video](https://www.youtube.com/watch?v=KeJINHjyzOU) at 15:30 or in [here](https://habrahabr.ru/post/275429/) but in 2D. You can google, search code at github, whatever, but the network should be based on Theano. 

Basically you will need to come up with true distribution $P$, say mixture of gaussians (surprise me), sample some data from it. Visualize it as a heatmap. To visualize $G$ density you can fix $N$ noise vectors $\{z_i\} \quad i=1,\dots, N$ and draw a circle for each $G(z_i)$. It is also funny to visualize discriminator as a vector field (can be done with `plt.arrow`, `plt.quiver plo). Look how it should be in the middle of [this page](http://www.inference.vc/an-alternative-update-rule-for-generative-adversarial-networks/).

Please, make sure your code works if 'Run All' is pressed and it draws some animation.

Good luck!

In [ ]:
import numpy as np 
import theano
import theano.tensor as T
import lasagne
import matplotlib.pyplot as plt
import scipy.stats as st
import sys

from tqdm import tqdm
from IPython.display import clear_output
from matplotlib import animation

%matplotlib inline

In [ ]:
def sample(params=[((4, -3), 4), ((-3, 1), 10)]):
    idx = np.random.choice(len(params))
    mean = params[idx][0]
    sigma = params[idx][1]
    cov_mat = [[sigma, 0], [0, sigma]]
    return np.random.multivariate_normal(mean, cov_mat)


def plot_dencity(samples, colors='k', limits=((-10, 10), (-10, 10))):
    x, y = samples[:, 0], samples[:, 1]
    x_min, x_max = limits[0]
    y_min, y_max = limits[1]
    
    xx, yy = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
    positions = np.vstack([xx.ravel(), yy.ravel()])
    kernel = st.gaussian_kde(samples.T)
    f = np.reshape(kernel(positions).T, xx.shape)
    
    ax = plt.gca()
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

    cset = ax.contour(xx, yy, f, colors=colors)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.show()

In [ ]:
samples = np.vstack([sample() for _ in range(10000)])
plot_dencity(samples)

In [ ]:
from lasagne.layers import DenseLayer, InputLayer
from lasagne.nonlinearities import rectify, LeakyRectify, sigmoid

g_samples = T.matrix("g_samples")
d_samples = T.matrix("d_samples")
targets = T.ivector("target")

DIM = 2

G = {}
G['input_layer'] = InputLayer((None, DIM), input_var=g_samples)
G['layer_1'] = DenseLayer(G['input_layer'], num_units=128, nonlinearity=LeakyRectify())
G['layer_2'] = DenseLayer(G['layer_1'], num_units=64, nonlinearity=LeakyRectify())
G['output_layer'] = DenseLayer(G['layer_2'], num_units=DIM, nonlinearity=None)

D = {}
D['gen_layer_1'] = DenseLayer(G['output_layer'], num_units=128, nonlinearity=rectify)
D['gen_layer_2'] = DenseLayer(D['gen_layer_1'], num_units=128, nonlinearity=rectify)
D['gen_output_layer'] = DenseLayer(D['gen_layer_2'], num_units=1, nonlinearity=sigmoid)

D['input_layer'] = InputLayer((None, DIM), input_var=d_samples)
D['layer_1'] = DenseLayer(D['input_layer'], num_units=128, nonlinearity=rectify, W=D['gen_layer_1'].W, b=D['gen_layer_1'].b)
D['layer_2'] = DenseLayer(D['layer_1'], num_units=128, nonlinearity=rectify, W=D['gen_layer_2'].W, b=D['gen_layer_2'].b)
D['output_layer'] = DenseLayer(D['layer_2'], num_units=1, nonlinearity=sigmoid, W=D['gen_output_layer'].W, b=D['gen_output_layer'].b)

In [ ]:
D_gen_prediction, D_prediction = lasagne.layers.get_output([D['gen_output_layer'], D['output_layer']], deterministic=True)
D_loss = T.mean(lasagne.objectives.binary_crossentropy(D_prediction, targets))
D_params = lasagne.layers.get_all_params(D['output_layer'], trainable=True)
D_updates = lasagne.updates.adam(D_loss, D_params)
D_train = theano.function([d_samples, targets], D_loss, updates=D_updates, allow_input_downcast = True)
D_predict = theano.function([d_samples], D_prediction, allow_input_downcast = True)

G_prediction = lasagne.layers.get_output(G['output_layer'], deterministic=True)
G_loss = -T.mean(T.log(D_gen_prediction))
G_params = lasagne.layers.get_all_params(G['output_layer'], trainable=True)
G_updates = lasagne.updates.adam(G_loss, G_params)
G_train = theano.function([g_samples], G_loss, updates=G_updates, allow_input_downcast = True)
G_predict = theano.function([g_samples], G_prediction, allow_input_downcast = True)

In [ ]:
def tran_step():
    for i in range(10):
        samples = np.zeros((BATCH_SIZE, DIM))
        samples[:int(BATCH_SIZE/2), :] = G_predict(np.random.rand(int(BATCH_SIZE/2), DIM))
        samples[int(BATCH_SIZE/2):, :] = np.vstack([sample() for _ in range(int(BATCH_SIZE/2))])
        labels = np.zeros(BATCH_SIZE)
        labels[int(BATCH_SIZE/2):] = 1
        d_loss = D_train(samples, labels)
        
        samples = np.random.rand(BATCH_SIZE, DIM)
        g_loss = G_train(samples)
        
        return g_loss, d_loss

    
def true_dencity():
    N_SAMPLES = 5000
    samples = np.vstack([sample() for _ in range(N_SAMPLES)])
    return st.gaussian_kde(samples.T)
    
    
def visualize(true_denc=true_dencity()):
    N_SAMPLES = 1000
    x_min, x_max = -10, 10
    y_min, y_max = -10, 10
    
    ax1.cla()
    ax2.cla()
    ax1.plot(*zip(*sorted(G_LOSS.items(),key=lambda t: t[0])), color="b")
    ax1.plot(*zip(*sorted(D_LOSS.items(),key=lambda t: t[0])), color="r")
    ax1.legend(["generator", "discriminator"])
    ax1.set_title("Loss")
    
    ax2.set_xlim(x_min, x_max)
    ax2.set_ylim(y_min, y_max)
    
    xx, yy = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
    positions = np.vstack([xx.ravel(), yy.ravel()])
    gen_samples = G_predict(np.random.rand(N_SAMPLES, DIM))
    gen_kernel = st.gaussian_kde(gen_samples.T)
    f = np.reshape(true_denc(positions).T, xx.shape)
    gen_f = np.reshape(gen_kernel(positions).T, xx.shape)
    ax2.contour(xx, yy, f, colors='k')
    ax2.contour(xx, yy, gen_f, colors='g')
    
    
    xx, yy = np.mgrid[x_min:x_max:25j, y_min:y_max:25j]
    positions = np.vstack([xx.ravel(), yy.ravel()])
    descr_T = D_predict(positions.T)
    descr_f = np.reshape(descr_T, xx.shape)
    
    u = np.cos(descr_f)
    v = np.sin(descr_f)
    m = np.hypot(u, v)
    ax2.quiver(xx, yy, u, v, m, units='x', pivot='tip')
    
    ax2.set_title("Distribution")

def train(i):
    n_iter = 100
    for j in range(n_iter):
        g_l, d_l = tran_step()
        G_LOSS[i * n_iter + j] = g_l
        D_LOSS[i * n_iter + j] = d_l
    
    print("iteration #{}" .format(i))
    visualize()

In [ ]:
N_EPOCH = 100
BATCH_SIZE = 512

G_LOSS = {}
D_LOSS = {}
        
for i in range(N_EPOCH):
    clear_output(True)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    train(i)
    plt.show()

In [ ]:
N_EPOCH = 100
BATCH_SIZE = 512

G_LOSS = {}
D_LOSS = {}
        
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
anim = animation.FuncAnimation(fig, train, frames=N_EPOCH)
anim.save('demoanimation.gif', writer='imagemagick', fps=2);